This is the example for SLoG-Net. For details and citation, please look at: 
Ye, Chang, and Gonzalo Mateos. "Learning to Identify Sources of Network Diffusion." 2022 30th European Signal Processing Conference (EUSIPCO). IEEE, 2022.

In [1]:
import numpy as np
from numpy import linalg as LA
import torch; torch.set_default_dtype(torch.float64)
import torch.nn as nn
import torch.optim as optim
import copy
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy import linalg
from timeit import default_timer as timer
import networkx as nx
import os
import pickle
import datetime
#### Import SLOG packakes
from SLoGexample import SLOGexperiment as SLOGexperiment
import torch
import os

In [2]:
## Determine whether use GPU or CPU
useGPU = True
if useGPU and torch.cuda.is_available():
    device = 'cuda:0'
    torch.cuda.empty_cache()
else:
    device = 'cpu'
# Notify:
device = torch.device(device)
print("Device selected: %s" % device)

Device selected: cpu


Parameters

In [3]:
### Model parameters
K = 5 # number of layers

filterTrainType = 'h' # The filter is generated for training.
# filterTrainType = 'g' # The inverse filter if generated for training.

### Simulation parameters
simuParas = {}
nTrain = 20000 # Number of training samples
P = 100 # Number of observations for one sample
batchsize = P # Batch size for training
nValid = P # Number of sample for validating
nTest = P # Number of sample for testing
nEpochs = 20
early_stop_criteria = 1e-5
### Data parameters
L = 5
alpha = 1.0

### Graph parameters

nNodes = 20 # Number of nodes
S = int(0.15*nNodes) # Number of sources per signal
C = nNodes # constrain constant
q = 2 # Number of constrain vectors for the SLOG-Net
simuParas['nNodes'] = nNodes
simuParas['S'] = S

graphType = 'ER' # Type of graph
graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
graphOptions['probIntra'] = 0.3 # Probability of drawing edges


## Filter type: g or h
filterType = 'h'
## Signal mode: gaussion or 1
signalMode = 'Gaussion'
## Train mode: Wt or default (Vdiag(\tilde{h})V^T)
trainMode = 'default'
## Filter mode: Wt or default (Vdiag(\tilde{h})V^T)
filterMode = 'default'

## Selection mode: random or nodes with top-N_C degree
selectMode = 'random'

## Noise level
noiseLevel = 0.0
noiseType = 'uniform'

simuParas['nTrain'] = nTrain
simuParas['batchsize'] = batchsize
simuParas['nValid'] = nValid
simuParas['nTest'] = nTest
simuParas['L'] = L
simuParas['noiseLevel'] = noiseLevel
simuParas['noiseType'] = noiseType
simuParas['filterType'] = filterType
simuParas['signalMode'] = signalMode
simuParas['trainMode'] = trainMode
simuParas['filterMode'] = filterMode
simuParas['selectMode'] = selectMode
simuParas['graphType'] = graphType
simuParas['alpha'] = alpha
simuParas['nEpochs'] = nEpochs
simuParas['device'] = device
simuParas['early_stop_criteria'] = early_stop_criteria

tMax = None # Maximum number of diffusion times (W^t for t < tMax)
tMax = 5
simuParas['tMax'] = tMax

## model parameters
modelParas = {}
modelParas['filterTrainType'] = filterTrainType
modelParas['C'] = C
modelParas['q'] = q
modelParas['K'] = K

## Experiment parameters
expParas = {}
expParas['nRealiz'] = 2



Experiment: ER graph



In [4]:
## q = 1
graphType = 'ER' # Type of graph
graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
graphOptions['probIntra'] = 0.3 # Probability of drawing edges

simuParas['graphType'] = graphType
simuParas['nNodes'] = 20
expParas['nRealiz'] = 5
simuParas['nEpochs'] = 30
modelParas['q'] = 2
P = 400
batchsize_slog = P
nValid = P
nTest = P
simuParas['batchsize'] = batchsize
simuParas['nValid'] = nValid
simuParas['nTest'] = nTest
simuParas['nTrain'] = 20000
simuParas['alpha'] = 1.0
simuParas['batchsize'] = P
simuParas['nValid'] = P
simuParas['nTest'] = P
simuParas['L'] = 3
result_1 = SLOGexperiment.slog_experiments(simuParas = simuParas,
                 graphOptions = graphOptions,
                 modelParas = modelParas, expParas = expParas)



Device selected: cpu
<class 'numpy.float64'>
(g_batch_generate) Generating h filter
[-0.22949808  0.23155383  0.22750287  0.2260739   0.22628462 -1.24179144
 12.07399801  3.73987664  1.1584915   0.23465335  0.23922697  0.24423482
  0.246551    0.26571375  0.57364098  0.3009196   0.31590079  0.42210633
  0.39288769  0.35167287]
[20000, 20000]
Number of Batches: 50

Epoch 1

    (E:  1, B:   1) 
	 Loss: 0.1406 [T] 0.1491 [V] 	 RE: 1.0004 [T] 	 RE: 0.9991 [V] 	 Best Loss: 0.1491 [V] 	 Loss: 0.1581 [Test]

    (E:  1, B:  41) 
	 Loss: 0.1453 [T] 0.1326 [V] 	 RE: 0.9993 [T] 	 RE: 0.9423 [V] 	 Best Loss: 0.1326 [V] 	 Loss: 0.1599 [Test]
Mean training time =  0.015978967999999937

Epoch 2

    (E:  2, B:  31) 
	 Loss: 0.1615 [T] 0.1424 [V] 	 RE: 1.0236 [T] 	 RE: 0.9763 [V] 	 Best Loss: 0.1326 [V] 	 Loss: 0.1676 [Test]
Mean training time =  0.015451053999999935

Epoch 3

    (E:  3, B:  21) 
	 Loss: 0.1535 [T] 0.1574 [V] 	 RE: 1.0387 [T] 	 RE: 1.0266 [V] 	 Best Loss: 0.1326 [V] 	 Loss: 0.1895 


    (E:  5, B:  41) 
	 Loss: 0.0161 [T] 0.0190 [V] 	 RE: 0.3291 [T] 	 RE: 0.3627 [V] 	 Best Loss: 0.0186 [V] 	 Loss: 0.2533 [Test]
Mean training time =  0.01675339399999956

Epoch 6

    (E:  6, B:  31) 
	 Loss: 0.0308 [T] 0.0163 [V] 	 RE: 0.4469 [T] 	 RE: 0.3352 [V] 	 Best Loss: 0.0163 [V] 	 Loss: 0.2120 [Test]
Mean training time =  0.01626818400000033

Epoch 7

    (E:  7, B:  21) 
	 Loss: 0.0651 [T] 0.0639 [V] 	 RE: 0.6611 [T] 	 RE: 0.6647 [V] 	 Best Loss: 0.0163 [V] 	 Loss: 0.2530 [Test]
Mean training time =  0.016605618000000107

Epoch 8

    (E:  8, B:  11) 
	 Loss: 0.0369 [T] 0.0314 [V] 	 RE: 0.5004 [T] 	 RE: 0.4662 [V] 	 Best Loss: 0.0163 [V] 	 Loss: 0.2409 [Test]
Mean training time =  0.016425289999999676

Epoch 9

    (E:  9, B:   1) 
	 Loss: 0.0040 [T] 0.0044 [V] 	 RE: 0.1627 [T] 	 RE: 0.1736 [V] 	 Best Loss: 0.0044 [V] 	 Loss: 0.2323 [Test]

    (E:  9, B:  41) 
	 Loss: 0.0070 [T] 0.0111 [V] 	 RE: 0.2130 [T] 	 RE: 0.2768 [V] 	 Best Loss: 0.0044 [V] 	 Loss: 0.2632 [Test]
Me

Mean training time =  0.016277081999999724

Epoch 12

    (E: 12, B:  11) 
	 Loss: 0.1236 [T] 0.1200 [V] 	 RE: 0.8846 [T] 	 RE: 0.8860 [V] 	 Best Loss: 0.0047 [V] 	 Loss: 0.2751 [Test]
Mean training time =  0.015918296000000113

Epoch 13

    (E: 13, B:   1) 
	 Loss: 0.0079 [T] 0.0072 [V] 	 RE: 0.2293 [T] 	 RE: 0.2168 [V] 	 Best Loss: 0.0047 [V] 	 Loss: 0.3560 [Test]

    (E: 13, B:  41) 
	 Loss: 0.0038 [T] 0.0035 [V] 	 RE: 0.1590 [T] 	 RE: 0.1509 [V] 	 Best Loss: 0.0035 [V] 	 Loss: 0.3601 [Test]
Mean training time =  0.01584294799999924

Epoch 14

    (E: 14, B:  31) 
	 Loss: 0.0066 [T] 0.0055 [V] 	 RE: 0.2072 [T] 	 RE: 0.1892 [V] 	 Best Loss: 0.0035 [V] 	 Loss: 0.3155 [Test]
Mean training time =  0.015935262000000422

Epoch 15

    (E: 15, B:  21) 
	 Loss: 0.0040 [T] 0.0033 [V] 	 RE: 0.1659 [T] 	 RE: 0.1465 [V] 	 Best Loss: 0.0033 [V] 	 Loss: 0.3181 [Test]
Mean training time =  0.016358987999999783

Epoch 16

    (E: 16, B:  11) 
	 Loss: 0.1072 [T] 0.1135 [V] 	 RE: 0.8579 [T] 	 RE: 0


Epoch 18

    (E: 18, B:  31) 
	 Loss: 0.0007 [T] 0.0017 [V] 	 RE: 0.0666 [T] 	 RE: 0.1057 [V] 	 Best Loss: 0.0010 [V] 	 Loss: 0.3151 [Test]
Mean training time =  0.01603642000000008

Epoch 19

    (E: 19, B:  21) 
	 Loss: 0.0012 [T] 0.0011 [V] 	 RE: 0.0892 [T] 	 RE: 0.0828 [V] 	 Best Loss: 0.0010 [V] 	 Loss: 0.3163 [Test]
Mean training time =  0.01542452800000092

Epoch 20

    (E: 20, B:  11) 
	 Loss: 0.0002 [T] 0.0007 [V] 	 RE: 0.0396 [T] 	 RE: 0.0653 [V] 	 Best Loss: 0.0007 [V] 	 Loss: 0.3144 [Test]
Mean training time =  0.0160857180000005

Epoch 21

    (E: 21, B:   1) 
	 Loss: 0.0016 [T] 0.0018 [V] 	 RE: 0.1038 [T] 	 RE: 0.1080 [V] 	 Best Loss: 0.0007 [V] 	 Loss: 0.3137 [Test]

    (E: 21, B:  41) 
	 Loss: 0.0034 [T] 0.0035 [V] 	 RE: 0.1507 [T] 	 RE: 0.1515 [V] 	 Best Loss: 0.0007 [V] 	 Loss: 0.3149 [Test]
Mean training time =  0.015626421999999706

Epoch 22

    (E: 22, B:  31) 
	 Loss: 0.0015 [T] 0.0014 [V] 	 RE: 0.0981 [T] 	 RE: 0.0969 [V] 	 Best Loss: 0.0007 [V] 	 Loss: 0.31

Mean training time =  0.015873403999999595

Epoch 25

    (E: 25, B:   1) 
	 Loss: 0.0068 [T] 0.0023 [V] 	 RE: 0.2070 [T] 	 RE: 0.1246 [V] 	 Best Loss: 0.0014 [V] 	 Loss: 0.2825 [Test]

    (E: 25, B:  41) 
	 Loss: 0.0020 [T] 0.0023 [V] 	 RE: 0.1161 [T] 	 RE: 0.1257 [V] 	 Best Loss: 0.0014 [V] 	 Loss: 0.2960 [Test]
Mean training time =  0.015924080000000913

Epoch 26

    (E: 26, B:  31) 
	 Loss: 0.0013 [T] 0.0021 [V] 	 RE: 0.0987 [T] 	 RE: 0.1191 [V] 	 Best Loss: 0.0014 [V] 	 Loss: 0.2956 [Test]
Mean training time =  0.016504446000000145

Epoch 27

    (E: 27, B:  21) 
	 Loss: 0.0013 [T] 0.0014 [V] 	 RE: 0.0913 [T] 	 RE: 0.0987 [V] 	 Best Loss: 0.0014 [V] 	 Loss: 0.3309 [Test]
Mean training time =  0.017386404000001788

Epoch 28

    (E: 28, B:  11) 
	 Loss: 0.0053 [T] 0.0063 [V] 	 RE: 0.1865 [T] 	 RE: 0.2057 [V] 	 Best Loss: 0.0014 [V] 	 Loss: 0.2967 [Test]
Mean training time =  0.016388714000001983

Epoch 29

    (E: 29, B:   1) 
	 Loss: 0.0047 [T] 0.0021 [V] 	 RE: 0.1742 [T] 	 RE: 

In [5]:
experiment_results = result_1.experiment_results
for exp_result in experiment_results:
  myModel = exp_result['model']
  print(exp_result['model'].saveDir)
  # print(exp_result['model'])
  print(exp_result['Graph'])
  # print(exp_result['training result'])


experiments\sourceLocSLOGNET-ER-20240818175346
experiments\sourceLocSLOGNET-ER-20240818175415
experiments\sourceLocSLOGNET-ER-20240818175445
experiments\sourceLocSLOGNET-ER-20240818175514
experiments\sourceLocSLOGNET-ER-20240818175544


In [6]:
save_test_result = True
N_realiz = expParas['nRealiz']
simuParas['N_realiz'] =  N_realiz
n0 = 0
d_noiseLvs = 0.05
N_noiseLvs = 2
N_model = len(experiment_results)
noiseLvs = n0 + d_noiseLvs*np.arange(N_noiseLvs)
result_exp_rex = np.zeros([N_noiseLvs, N_model,N_realiz])
result_exp_acc = np.zeros([N_noiseLvs, N_model,N_realiz])
result_exp_reg = np.zeros([N_noiseLvs, N_model,N_realiz])
result_elapse = np.zeros([N_noiseLvs, N_model,N_realiz])
n_model = 0
for exp_result in experiment_results:
    for n_nlvs in range(N_noiseLvs):
        simuParas['noiseLevel'] = noiseLvs[n_nlvs]
        print('Model ', n_model, ', in ', exp_result['model'].saveDir,',noiseLevel = ', noiseLvs[n_nlvs])
        result = SLOGexperiment.test_local(nNodes,P,S, exp_result, simuParas = simuParas,modelParas = modelParas)
        result_exp_rex[n_nlvs,n_model,:] = result['re_x']
        result_exp_acc[n_nlvs,n_model,:] = result['acc_x']
        result_exp_reg[n_nlvs,n_model,:] = result['re_g']
        result_elapse[n_nlvs,n_model,:] = result['elapse']
    n_model += 1
test_result = {}
test_result['result_exp_rex'] = result_exp_rex
test_result['result_exp_acc'] = result_exp_acc
test_result['result_exp_reg'] = result_exp_reg
test_result['result_elapse'] = result_elapse
test_info = {}
test_info['nTrain'] = nTrain
test_info['graphType'] = graphType
test_result['test_info'] = test_info

Model  0 , in  experiments\sourceLocSLOGNET-ER-20240818175346 ,noiseLevel =  0.0
Model  0 , in  experiments\sourceLocSLOGNET-ER-20240818175346 ,noiseLevel =  0.05
Model  1 , in  experiments\sourceLocSLOGNET-ER-20240818175415 ,noiseLevel =  0.0
Model  1 , in  experiments\sourceLocSLOGNET-ER-20240818175415 ,noiseLevel =  0.05
Model  2 , in  experiments\sourceLocSLOGNET-ER-20240818175445 ,noiseLevel =  0.0
Model  2 , in  experiments\sourceLocSLOGNET-ER-20240818175445 ,noiseLevel =  0.05
Model  3 , in  experiments\sourceLocSLOGNET-ER-20240818175514 ,noiseLevel =  0.0
Model  3 , in  experiments\sourceLocSLOGNET-ER-20240818175514 ,noiseLevel =  0.05
Model  4 , in  experiments\sourceLocSLOGNET-ER-20240818175544 ,noiseLevel =  0.0
Model  4 , in  experiments\sourceLocSLOGNET-ER-20240818175544 ,noiseLevel =  0.05


In [7]:
print(result_exp_rex[0,:,:])
print(result_exp_reg[0,:,:])

[[0.10887897 0.15561489 0.0844307  0.10655521 0.23604117]
 [0.08905913 0.15263509 0.07920018 0.06879606 0.11979194]
 [1.5711588  0.06976589 1.01885255 1.55430064 0.0947757 ]
 [0.05900903 1.44049835 0.04985803 0.07637001 0.07002528]
 [0.07990917 0.1075338  0.11010116 0.11792574 0.10871998]]
[[0.11009362 0.1602508  0.08094347 0.09819387 0.35987357]
 [0.07800981 0.15922789 0.07665857 0.06686606 0.10499347]
 [1.09706911 0.07210426 0.86456034 0.89693856 0.08619165]
 [0.06430009 0.32276134 0.05118087 0.07616097 0.08639812]
 [0.08280749 0.10846423 0.10781324 0.11499009 0.11937308]]
